In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from builtins import input

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ip_1 = pd.read_csv("/content/drive/MyDrive/seating/ip_1.csv", header=1)
ip_2 = pd.read_csv("/content/drive/MyDrive/seating/ip_2.csv", header=1)
ip_3 = pd.read_csv("/content/drive/MyDrive/seating/ip_3.csv", header=0)
ip_4 = pd.read_csv("/content/drive/MyDrive/seating/ip_4.csv", header=0)

In [ ]:
ip_1.head()

,rollno,register_sem,schedule_sem,course_code
0,1401MC57,4,4,HS202
1,1401MC57,4,4,CS244
2,1401MC57,4,4,MA216
3,1401MC57,4,4,MA218
4,1401MC57,4,4,MA220


In [ ]:
ip_1['course_code'] = ip_1['course_code'].str.strip()

In [ ]:
ip_2.head()

,Date,Day,Morning,Evening
0,30/04/2016,Sunday,CS249; CH426; MM304; CB308; CE216; CB204; PH42...,NO EXAM
1,01/05/2016,Monday,HS202; HS212; HS225; HS302; HS312; HS341; HS33...,CS514; CE512; CH606; ME526; ME522; EE513; EE58...
2,02/05/2016,Tuesday,HS514; CS372; CS392; CH422; EP206; PH201; CS2...,MA504; CE506; MH502; EE507; MM502; CB504; CS54...
3,03/05/2016,Wednesday,CS259; MA428; MA220; CE218; CH428; ME216; EE20...,CS551; CE514; ME542; ME544; EE514; EE549; PH60...
4,04/05/2016,Thursday,HS716; CS209; MA430; CE220; ME214; EE280; EE38...,CS547; CS568; CE545; CH607; ME567; ME547; ME55...


In [ ]:
ip_3.head()

,Room No.,Exam Capacity,Block
0,101,30,9
1,102,72,9
2,103,30,9
3,104,72,9
4,105,30,9


In [ ]:
ip_4.head()

,Roll,Name
0,1121CH05,Ajit Singh
1,1121CH07,Jaishree Mayank
2,1121CH08,Sudipta Acharya
3,1121CH11,Nilesh Chakraborty
4,1121CH12,Mohammad Junaid Akhtar


In [ ]:
ip_1.course_code.value_counts()

,count
course_code,
CS244,290
HS202,284
MA225,268
CS551,173
HS331,151
...,...
CS226,1
CS299,1
EE518,1


In [ ]:
ip_3

,Room No.,Exam Capacity,Block
0,101,30,9
1,102,72,9
2,103,30,9
3,104,72,9
4,105,30,9
5,106,30,9
6,107,72,9
7,108,30,9
8,109,72,9
9,110,30,9


In [ ]:
def extract_floor(room_no):
    if room_no[:2].isalpha():
        return room_no[:]
    else:
        return int(room_no[0])

ip_3['floor'] = ip_3['Room No.'].apply(extract_floor)

In [ ]:
room_cap = ip_3.sort_values(by=['floor', 'Exam Capacity'], ascending=[True, True])
room_cap

,Room No.,Exam Capacity,Block,floor
0,101,30,9,1
2,103,30,9,1
4,105,30,9,1
5,106,30,9,1
7,108,30,9,1
9,110,30,9,1
1,102,72,9,1
3,104,72,9,1
6,107,72,9,1
8,109,72,9,1


In [ ]:
room_cap.head()

,Room No.,Exam Capacity,Block,floor
0,101,30,9,1
2,103,30,9,1
4,105,30,9,1
5,106,30,9,1
7,108,30,9,1


In [ ]:
course_counts = ip_1['course_code'].value_counts().reset_index()

course_counts.columns = ['course_code', 'count']

course_counts.head()

,course_code,count
0,CS244,290
1,HS202,284
2,MA225,268
3,CS551,173
4,HS331,151


In [ ]:
def result(course_counts, room_cap, buffer):

  allocations = []

  for _, course in course_counts.iterrows():
      course_code = course["course_code"]
      remaining_students = course["count"]
      course_allocations = []

      for i, room in room_cap.iterrows():
          room_capacity = room["Exam Capacity"]
          effective_capacity = room["effective_capacity"]

          if remaining_students <= 0:
              break

          if effective_capacity > 0:
              if remaining_students <= effective_capacity:
                  course_allocations.append({
                      "course_code": course_code,
                      "room_no": room["Room No."],
                      "num_students_allocated": remaining_students,
                      "vacant_seats": effective_capacity - remaining_students,
                      "buffered_capacity": room["Exam Capacity"] * buffer
                  })
                  # Update room effective capacity
                  room_cap.at[i, "effective_capacity"] -= remaining_students
                  remaining_students = 0
              else:
                  course_allocations.append({
                      "course_code": course_code,
                      "room_no": room["Room No."],
                      "num_students_allocated": effective_capacity,
                      "vacant_seats": 0,
                      "buffered_capacity": room["Exam Capacity"] * buffer
                  })
                  remaining_students -= effective_capacity
                  room_cap.at[i, "effective_capacity"] = 0

      if remaining_students > 0:
          course_allocations.append({
              "course_code": course_code,
              "room_no": None,
              "num_students_allocated": remaining_students,
              "vacant_seats": 0,
              "buffered_capacity": 0
          })

      allocations.extend(course_allocations)

  allocations_df = pd.DataFrame(allocations)

  return allocations_df

In [ ]:
final = pd.DataFrame()

In [ ]:
# result_df['course_code'].value_counts()

In [ ]:
buffer = int(input("write the amount of buffer you want in room in percentage (e.g., 10):  "))/100

write the amount of buffer you want in room in percentage (e.g., 10):  5


In [ ]:
room_cap["effective_capacity"] = (room_cap["Exam Capacity"] * (1 - buffer)).astype(int)

for i, row in ip_2.iterrows():
  mor = pd.DataFrame({'course_code': row['Morning'].split('; ')})
  mor['course_code'] = mor['course_code'].str.strip()
  mor['count'] =  mor['course_code'].apply(lambda code: ip_1[ip_1['course_code'] == code].shape[0])

  mor = mor.sort_values(by="count", ascending=False).reset_index(drop=True)

  result_df = result(mor, room_cap, buffer)
  final = pd.concat([final, result_df], ignore_index=True)
  room_cap["effective_capacity"] = (room_cap["Exam Capacity"] * (1 - buffer)).astype(int)

  eve = pd.DataFrame({'course_code': row['Evening'].split('; ')})
  eve['course_code'] = eve['course_code'].str.strip()
  eve['count'] =  eve['course_code'].apply(lambda code: ip_1[ip_1['course_code'] == code].shape[0])

  eve = eve.sort_values(by="count", ascending=False).reset_index(drop=True)

  result_df = result(eve, room_cap, buffer)
  final = pd.concat([final, result_df], ignore_index=True)
  room_cap["effective_capacity"] = (room_cap["Exam Capacity"] * (1 - buffer)).astype(int)


In [ ]:
final.sample(15)

,course_code,room_no,num_students_allocated,vacant_seats,buffered_capacity
174,ME544,107,17,51,3.6
331,CS508,107,11,0,3.6
168,CS551,108,28,0,1.5
322,CE528,108,6,0,1.5
31,HS341,306,21,0,1.5
155,MA428,303,14,54,3.6
97,CH422,310,14,14,1.5
53,EE320,110,28,0,1.5
325,EE529,102,46,0,3.6
187,CS209,101,28,0,1.5


In [ ]:
final[final['course_code']=='MA424']

,course_code,room_no,num_students_allocated,vacant_seats,buffered_capacity
92,MA424,306,14,0,1.5
93,MA424,307,16,12,1.5


In [ ]:
op_1 = pd.DataFrame(columns=["Date","Course", "Shift", "Room", "Students"])

for index, row in ip_2.iterrows():

  mor = pd.DataFrame({'Course': row['Morning'].split('; '), 'Shift': 'Morning', 'Date': row['Date']})
  op_1 = pd.concat([op_1, mor], ignore_index=True)
  first = row['Morning'].split('; ')

  eve = pd.DataFrame({'Course': row['Evening'].split('; '), 'Shift': 'Evening', 'Date': row['Date']})
  op_1 = pd.concat([op_1, eve], ignore_index=True)

op_1 = op_1.explode('Course').reset_index(drop=True)

In [ ]:
op_1.head()

,Date,Course,Shift,Room,Students
0,30/04/2016,CS249,Morning,NaN,NaN
1,30/04/2016,CH426,Morning,NaN,NaN
2,30/04/2016,MM304,Morning,NaN,NaN
3,30/04/2016,CB308,Morning,NaN,NaN
4,30/04/2016,CE216,Morning,NaN,NaN


In [ ]:
final['course_code'] = final['course_code'].str.strip()
op_1['Course'] = op_1['Course'].str.strip()
expanded_op_1 = op_1.merge(final[['course_code', 'room_no', 'num_students_allocated','buffered_capacity']],
                           left_on='Course', right_on='course_code', how='left')

expanded_op_1 = expanded_op_1.drop(columns=['course_code'])

expanded_op_1 = expanded_op_1.reset_index(drop=True)

expanded_op_1

,Date,Course,Shift,Room,Students,room_no,num_students_allocated,buffered_capacity
0,30/04/2016,CS249,Morning,NaN,NaN,110,4.0,1.5
1,30/04/2016,CS249,Morning,NaN,NaN,102,36.0,3.6
2,30/04/2016,CH426,Morning,NaN,NaN,104,11.0,3.6
3,30/04/2016,CH426,Morning,NaN,NaN,107,15.0,3.6
4,30/04/2016,MM304,Morning,NaN,NaN,104,29.0,3.6
...,...,...,...,...,...,...,...,...
329,06/05/2016,CS563,Evening,NaN,NaN,103,28.0,1.5
330,06/05/2016,CS563,Evening,NaN,NaN,105,28.0,1.5
331,06/05/2016,CS563,Evening,NaN,NaN,106,28.0,1.5
332,06/05/2016,CS563,Evening,NaN,NaN,108,22.0,1.5


In [ ]:
expanded_op_1.drop(columns=['Room', 'Students'], inplace=True)

In [ ]:
expanded_op_1['Date'] = pd.to_datetime(expanded_op_1['Date'], format='%d/%m/%Y')

In [ ]:
expanded_op_1.sort_values(by=['Date', 'Shift', 'Course', 'room_no'], ascending=[True, False, True, True], inplace=True)

In [ ]:
expanded_op_1['Date'] = expanded_op_1['Date'].dt.strftime('%d/%m/%Y')

In [ ]:
expanded_op_1

,Date,Course,Shift,room_no,num_students_allocated,buffered_capacity
10,30/04/2016,CB204,Morning,101,28.0,1.5
11,30/04/2016,CB204,Morning,103,28.0,1.5
5,30/04/2016,CB308,Morning,105,28.0,1.5
6,30/04/2016,CB308,Morning,106,26.0,1.5
7,30/04/2016,CE216,Morning,106,2.0,1.5
...,...,...,...,...,...,...
318,06/05/2016,EE529,Evening,102,46.0,3.6
319,06/05/2016,EE529,Evening,104,8.0,3.6
326,06/05/2016,EE573,Evening,104,7.0,3.6
327,06/05/2016,EE573,Evening,107,40.0,3.6


In [ ]:
bachhe = pd.DataFrame(columns=["Course", "Students"])
courses = []
students = []

for group, group_df in ip_1.groupby('course_code'):
  courses.append(group)  # Add course code
  student_list = ', '.join(group_df['rollno'])  # Combine roll numbers as a string
  students.append(student_list)

bachhe['Course'] = courses
bachhe['Students'] = students

In [ ]:
bachhe

,Course,Students
0,CB202,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
1,CB204,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
2,CB206,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
3,CB208,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
4,CB210,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
...,...,...
227,PH602,"1412PH23, 1412PH19, 1412PH29, 1412PH04, 1412PH..."
228,PH603,"1412PH23, 1412PH19, 1412PH29, 1412PH03, 1412PH..."
229,PH605,"1412PH22, 1412PH18, 1412PH05, 1412PH24, 1412PH..."
230,PH608,"1412PH23, 1412PH22, 1412PH18, 1412PH05, 1412PH..."


In [ ]:
bachhe_sorted = bachhe.copy()
bachhe_sorted['Students'] = bachhe_sorted['Students'].apply(lambda students: ', '.join(sorted(students.split(', '))))
bachhe_sorted

,Course,Students
0,CB202,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
1,CB204,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
2,CB206,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
3,CB208,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
4,CB210,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
...,...,...
227,PH602,"1412PH02, 1412PH04, 1412PH15, 1412PH16, 1412PH..."
228,PH603,"1412PH03, 1412PH15, 1412PH16, 1412PH17, 1412PH..."
229,PH605,"1412PH01, 1412PH02, 1412PH04, 1412PH05, 1412PH..."
230,PH608,"1412PH01, 1412PH02, 1412PH03, 1412PH04, 1412PH..."


In [ ]:
expanded_op_1['num_students_allocated'].fillna(0, inplace=True)
expanded_op_1['buffered_capacity'].fillna(0, inplace=True)

<ipython-input-160-865ca2865ac2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  expanded_op_1['num_students_allocated'].fillna(0, inplace=True)
<ipython-input-160-865ca2865ac2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

In [ ]:
expanded_op_1_sorted = expanded_op_1.copy()

In [ ]:
expanded_op_1['log'] = ''

for index, row in expanded_op_1.iterrows():
    course = row['Course']
    no_of_students = int(row['num_students_allocated'])

    bachhe_row = bachhe[bachhe['Course'] == course]

    if not bachhe_row.empty:
          students_list = bachhe_row.iloc[0]['Students']

          if isinstance(students_list, str):
              students_list = students_list.split(', ')

          selected = students_list[:no_of_students]

          expanded_op_1.at[index, 'log'] = ', '.join(selected)

          # Remove the selected students from bachhe
          remaining_students = students_list[no_of_students:]
          bachhe.loc[bachhe['Course'] == course, 'Students'] = ', '.join(remaining_students)


In [ ]:
expanded_op_1_sorted['log'] = ''

for index, row in expanded_op_1_sorted.iterrows():
    course = row['Course']
    no_of_students = int(row['num_students_allocated'])

    bachhe_row = bachhe_sorted[bachhe_sorted['Course'] == course]

    if not bachhe_row.empty:
          students_list = bachhe_row.iloc[0]['Students']

          if isinstance(students_list, str):
              students_list = students_list.split(', ')

          selected = students_list[:no_of_students]

          expanded_op_1_sorted.at[index, 'log'] = ', '.join(selected)

          # Remove the selected students from bachhe
          remaining_students = students_list[no_of_students:]
          bachhe_sorted.loc[bachhe_sorted['Course'] == course, 'Students'] = ', '.join(remaining_students)


In [ ]:
# seating is done randomly
expanded_op_1

,Date,Course,Shift,room_no,num_students_allocated,buffered_capacity,log
10,30/04/2016,CB204,Morning,101,28.0,1.5,"1401CB54, 1401CB22, 1401CB21, 1401CB19, 1401CB..."
11,30/04/2016,CB204,Morning,103,28.0,1.5,"1401CB66, 1401CB16, 1401CB41, 1401CB64, 1401CB..."
5,30/04/2016,CB308,Morning,105,28.0,1.5,"1301CB46, 1301CB26, 1301CB33, 1301CB57, 1301CB..."
6,30/04/2016,CB308,Morning,106,26.0,1.5,"1301CB62, 1301CB35, 1301CB11, 1301CB43, 1301CB..."
7,30/04/2016,CE216,Morning,106,2.0,1.5,"1401CE53, 1401CE24"
...,...,...,...,...,...,...,...
318,06/05/2016,EE529,Evening,102,46.0,3.6,"1201EE29, 1201EE43, 1201EE72, 1201EE56, 1201EE..."
319,06/05/2016,EE529,Evening,104,8.0,3.6,"1201EE20, 1511EE11, 1511EE04, 1201EE51, 1511EE..."
326,06/05/2016,EE573,Evening,104,7.0,3.6,"1201EE35, 1201EE23, 1201EE16, 1511EE14, 1201EE..."
327,06/05/2016,EE573,Evening,107,40.0,3.6,"1511EE22, 1201EE66, 1511EE37, 1511EE42, 1511EE..."


In [ ]:
# seating by branch wise
expanded_op_1_sorted

,Date,Course,Shift,room_no,num_students_allocated,buffered_capacity,log
10,30/04/2016,CB204,Morning,101,28.0,1.5,"1401CB01, 1401CB02, 1401CB03, 1401CB05, 1401CB..."
11,30/04/2016,CB204,Morning,103,28.0,1.5,"1401CB36, 1401CB37, 1401CB38, 1401CB39, 1401CB..."
5,30/04/2016,CB308,Morning,105,28.0,1.5,"1301CB01, 1301CB02, 1301CB03, 1301CB04, 1301CB..."
6,30/04/2016,CB308,Morning,106,26.0,1.5,"1301CB35, 1301CB36, 1301CB37, 1301CB38, 1301CB..."
7,30/04/2016,CE216,Morning,106,2.0,1.5,"1401CE03, 1401CE04"
...,...,...,...,...,...,...,...
318,06/05/2016,EE529,Evening,102,46.0,3.6,"1201EE03, 1201EE05, 1201EE07, 1201EE09, 1201EE..."
319,06/05/2016,EE529,Evening,104,8.0,3.6,"1511EE38, 1511EE39, 1511EE44, 1511EE46, 1511EE..."
326,06/05/2016,EE573,Evening,104,7.0,3.6,"1201EE02, 1201EE04, 1201EE06, 1201EE08, 1201EE..."
327,06/05/2016,EE573,Evening,107,40.0,3.6,"1201EE16, 1201EE17, 1201EE18, 1201EE19, 1201EE..."


In [ ]:
file_path = "output.xlsx"
expanded_op_1.to_excel(file_path, index=False)

In [ ]:
file_path = "output_branchwise.xlsx"
expanded_op_1_sorted.to_excel(file_path, index=False)

In [ ]:
ip_4.head()

,Roll,Name
0,1121CH05,Ajit Singh
1,1121CH07,Jaishree Mayank
2,1121CH08,Sudipta Acharya
3,1121CH11,Nilesh Chakraborty
4,1121CH12,Mohammad Junaid Akhtar


In [ ]:
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Border, Side

In [ ]:
# Create attendance sheet for each room and shift
for _, rows in expanded_op_1.iterrows():
    date = rows["Date"]
    shift = rows["Shift"]
    courses = rows["Course"]
    rooms = rows["room_no"]
    studnts = rows["log"]
    studnts = studnts.split(', ')
    studnts = [student.strip() for student in studnts]

    # Create attendance DataFrame
    attendance_sheet = pd.DataFrame()
    attendance_sheet["Stud_Roll"] = studnts
    attendance_sheet["Stud_Name"] = attendance_sheet["Stud_Roll"].map(ip_4.set_index("Roll")["Name"])
    attendance_sheet["Sign"] = ""

    # Add blank rows for signatures
    empty_df = pd.DataFrame({"Stud_Roll": [""] * 5, "Stud_Name": [""] * 5, "Sign": [""] * 5})
    attendance_sheet = pd.concat([attendance_sheet, empty_df], ignore_index=True)

    # Save as Excel file
    sanitized_date = date.replace("/", "-")  # Replace / with -
    file_name = f"{sanitized_date}_{shift}_{courses}_Room_{rooms}.xlsx"

    wb = openpyxl.Workbook()
    ws = wb.active
    # give title to the worksheet
    ws.title = f"{courses} Room {rooms}"

    # Write the attendance data to the worksheet
    for r in dataframe_to_rows(attendance_sheet, index=False, header=True):
        ws.append(r)

    # Adjust column width based on name length
    max_name_length = max(attendance_sheet["Stud_Name"].str.len())
    ws.column_dimensions["B"].width = max(15, max_name_length + 2)

    # Apply alignment and border styles
    for row in ws.iter_rows():
        for box in row:
            box.alignment = Alignment(horizontal="center", vertical="center")
            box.border = Border(left=Side(style="thin"),
                                 right=Side(style="thin"),
                                 top=Side(style="thin"),
                                 bottom=Side(style="thin"))
    wb.save(file_name)
print("Attendance sheets are generated successfully.")

Attendance sheets are generated successfully.
